# This is a small mod of the Newaetech tutorial Introduction to Voltage Glitchin for CW-Nano
The sole intent for this mod is to enable the students at [University of South-eastern Noway](www.usn.no) to be able to run the example on the lab \
Please check [https://github.com/newaetech](https://github.com/newaetech) for utdates :) \
[original src:](https://github.com/newaetech/chipwhisperer-jupyter/blob/c940073159c8032877e9f7b9ef852b3662c4ec02/courses/fault101/SOLN_Fault%202_1B%20-%20Introduction%20to%20Voltage%20Glitching%20with%20CWNano.ipynb)
The examle could be runned using jupyter but requres some python libraries like: chipwhisperer , matplotlib ...

# Part 2, Topic 2: Introduction to Voltage Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *While it's not as sophisticated as the ChipWhisperer Lite or ChipWhisperer Pro's glitch hardware, the ChipWhisperer Nano is also capable of glitching. In this lab, we'll do some simple glitch tests on the Nano's target board, showing how to scan through glitch settings and seeing what effect it has on the hardware.*

**LEARNING OUTCOMES:**

* Understanding how voltage glitching can be used to disrupt a target's operation
* Scanning glitch settings to determine successful ones

Digital hardware devices have certain voltage and clock requirements to function properly. If these requirements are not met, the device can fail to function, or even be damage. By shorting the voltage pins of a microcontroller for controlled, short periods of time, we can cause it to behave erratically, clearning registers and skipping instructions.  Such attacks can be immensely powerful in practice. Consider for example the following code from [newaetech - simpleserial-glitch.c ](https://github.com/newaetech/chipwhisperer/blob/develop/hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch.c) :

```C
/*src: https://github.com/newaetech/chipwhisperer/blob/develop/hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch.c*/
/*Line 32 .. (+ small mod..)*/
uint8_t glitch_loop(uint8_t* in)
{
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    cnt = 0;
    
    trigger_high();        /* Chipwhisper starts VDD Glitching    */
                           /* One of the output pins is set high  */

    /* Glitch Victim Code */
    for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
   
    trigger_low();        /* Chipwhisper Stop VDD Glitching (Dont kill the serial output..) */
    
                                              /* Python listen for serial data using        */
                                              /* val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=100)  */
    simpleserial_put('r', 4, (uint8_t*)&cnt); /* Python reieves val['payload']             */

                                              /* Validate the result before exiting      */
    return (cnt != 2500);                     /* Python recieves val['valid'] True/False */
```

### Glitch Hardware

The ChipWhisperer Nano's glitch setup is pretty simple. Like its bigger brothers, the Lite and the Pro, it uses a MOSFET to short the microcontroller's voltage supply to ground:
Some new chip-whisper nano cards needs to be modifyed to produce sucessfull glitching.
( The new resistor was 5.6 Ohm [more info on github](https://github.com/newaetech/chipwhisperer/issues/419#issuecomment-1262053220)

![Original vs Modifyed CW-Nano](./mod_glitch.png)

For the Nano, `Glitch In` is controlled by 2 parameters:

1. `scope.glitch.ext_offset` - The glitch will be inserted roughly `8.3ns * scope.glitch.ext_offset`
1. `scope.glitch.repeat`     - The glitch will be inserted for roughly `8.3ns * scope.glitch.repeat`

During this lab, we'll be varying these parameters to see if we can get the target to mess up a calculation that it's doing.

In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "Setup_Generic.ipynb"


Program the Victim microcontroller STM32F04xxx <br>
using .hex output from the the C - file :  **[simpleserial-glitch.c](http://localhost:8888/edit/simpleserial-glitch.c)** 


In [ ]:
fw_path="simpleserial-glitch-CWNANO.hex"
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.io.clkout = 7.5E6
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()

In [ ]:
scope

In [ ]:
import struct
reboot_flush()
scope.glitch.repeat = 160
scope.glitch.ext_offset =100

scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=100)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']
    gcnt = struct.unpack("<I", val['payload'])[0]
    print("gcnt:" , gcnt)
print(val)

In [ ]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset"])
gc.display_stats()

Some tips for finding good glitches:

1. This is a VCC line that we're shorting, so there's going to be stuff fighting against us. If your glitch is too short, it might not have any effect
1. Likewise, if your glitch is too long, the target will always crash. There's typically a small band where you're able to affect the target, but it won't always crash it.
1. Be patient. Glitching can be somewhat inconsistant, so don't be discouraged if it takes a while to see some success!

In [ ]:
%matplotlib ipympl 
import matplotlib.pylab as plt
fig = plt.figure()

In [ ]:
fig.canvas.draw()

In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct
import time

g_step = 5

gc.set_global_step(g_step)
gc.set_range("repeat", 1, 60)
gc.set_range("ext_offset", 1, 400)
scope.glitch.repeat = 0

reboot_flush()
sample_size = 1
for glitch_setting in gc.glitch_values():
    target.flush()
    reboot_flush

    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]

    successes = 0
    resets = 0
    
    for i in range(3):
            
        scope.arm()
        #Do glitch loop
        target.simpleserial_write("g", bytearray([]))
        
        ret = scope.capture()
        
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
        
        if ret:
            print('Timeout - no trigger')
            gc.add("reset", (scope.glitch.repeat, scope.glitch.ext_offset))
            plt.plot(scope.glitch.ext_offset, scope.glitch.repeat, 'xr', alpha=1)
            fig.canvas.draw()
            resets += 1

            #Device is slow to boot?
            reboot_flush()

        else:
            if val['valid'] is False:
                reboot_flush()
                gc.add("reset", (scope.glitch.repeat, scope.glitch.ext_offset))
                plt.plot(scope.glitch.ext_offset, scope.glitch.repeat, 'xr', alpha=1)
                fig.canvas.draw()
                resets += 1
            else:
                gcnt = struct.unpack("<I", val['payload'])[0]
                
                if gcnt != 2500: #for loop check
                    gc.add("success", (scope.glitch.repeat, scope.glitch.ext_offset))
                    print(gcnt)
                    plt.plot(scope.glitch.ext_offset, scope.glitch.repeat, '+g', alpha=1)
                    fig.canvas.draw()
                    successes += 1
                else:
                    gc.add("normal", (scope.glitch.repeat, scope.glitch.ext_offset))
                    
    if successes > 0:                
        print("successes = {}, resets = {}, repeat = {}, ext_offset = {}".format(successes, resets, scope.glitch.repeat, scope.glitch.ext_offset))
print("Done glitching")

In [ ]:
%matplotlib notebook
gc.results.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
scope.dis()
target.dis()

Unlike the other ChipWhisperers, the Nano doesn't have sychronous glitching. This means that `ext_offset` is a mixture of both the offset within the clock cycle, which affects glitch success, and ext_offset, which affects which instruction is being glitched. As such, ext_offset settings you find in this lab won't be directly applicable to other labs. That being said, good ranges for repeat and the success rate of glitches still gives valuable information that you can apply to other labs.